In [1]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00


In [2]:
from pyvi import ViTokenizer
import tensorflow as tf
import string
import os
from tqdm import tqdm
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/ChatBox

/content/drive/MyDrive/ChatBox


In [5]:
%ls

data/  model/  src/


## **Data Pre-Processing**

In [6]:
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc)
    # doc = doc.lower() # lower
    tokens = doc.split() # split into words
    table = str.maketrans('', '', string.punctuation.replace("_", "")) # remove all punctuations
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]
    return tokens

# print(clean_document('đường phố ở việt nam'))

In [7]:
# Đường dẫn tới file CSV
TRAIN_PATH = './data/cleaned_articles.csv'

# Đọc file CSV
data = pd.read_csv(TRAIN_PATH)

In [8]:
data.head()

,title,content,subtopic
0,điều 1 phạm_vi điều_chỉnh,luật này quy_định về chế_độ sở_hữu đất_đai quy...,Luat-Dat-dai-2024
1,điều 2 đối_tượng áp_dụng,1 cơ_quan nhà_nước thực_hiện quyền_hạn và trác...,Luat-Dat-dai-2024
2,điều 2 đối_tượng áp_dụng,2 người sử_dụng đất,Luat-Dat-dai-2024
3,điều 2 đối_tượng áp_dụng,3 các đối_tượng khác có liên_quan đến việc quả...,Luat-Dat-dai-2024
4,điều 3 giải_thích từ_ngữ,trong luật này các từ_ngữ dưới đây được hiểu n...,Luat-Dat-dai-2024


In [9]:


INPUT_LENGTH = 50
sequences = []

# Duyệt qua từng dòng trong DataFrame
for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    doc = row['content']  # Giả sử cột chứa văn bản là 'article'
    tokens = clean_document(doc)
    # Tạo các chuỗi con từ token
    for i in range(0, len(tokens), 51):
        seq = tokens[i:i+51]
        line = ' '.join(seq)
        sequences.append(line)

100%|██████████| 12777/12777 [00:11<00:00, 1091.05it/s]


In [10]:
print('Total of senetences: ', len(sequences))
sequences[0:5]

Total of senetences:  14862


['luật này quy_định về chế_độ sở_hữu đất_đai quyền_hạn và trách_nhiệm của nhà_nước_đại_diện chủ sở_hữu toàn dân về đất_đai và thống_nhất quản_lý về đất_đai chế_độ quản_lý và sử_dụng đất_đai quyền và nghĩa_vụ của công_dân người sử_dụng đất đối_với đất_đai thuộc lãnh_thổ của nước cộng_hòa xã_hội chủ_nghĩa việt_nam',
 '1 cơ_quan nhà_nước thực_hiện quyền_hạn và trách_nhiệm đại_diện chủ sở_hữu toàn dân về đất_đai thực_hiện nhiệm_vụ thống_nhất quản_lý nhà_nước về đất_đai',
 '2 người sử_dụng đất',
 '3 các đối_tượng khác có liên_quan đến việc quản_lý sử_dụng đất_đai',
 'trong luật này các từ_ngữ dưới đây được hiểu như sau']

In [11]:
# # filter several punctuations in 50803 sentences
# tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!“"”#$%&()*+,-./:;<=>?@[\]^`{|}~ ')
# # create vocabulary (corpus) for each word in N sentences
# tokenizer.fit_on_texts(sequences)

# print(len(tokenizer.word_index))
# tokenizer.word_index

In [12]:
from transformers import AutoTokenizer, TFAutoModel

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [13]:
def create_input(text_data, max_length):
    input_ids = []
    for line in text_data:
        # Tokenize câu văn bản và chuyển đổi thành input_ids
        encoded_input = tokenizer(line, return_tensors='tf', max_length=max_length, padding='max_length', truncation=True)['input_ids']
        input_ids.append(encoded_input)
    return input_ids

In [14]:
max_length = max(len(tokenizer.encode(sequence)) for sequence in sequences)
input_ids = create_input(sequences, max_length)

# Stack các tensor input_ids lại với nhau
input_ids = tf.concat(input_ids, axis=0)
print('Shape of input_ids:', input_ids.shape)

Shape of input_ids: (14862, 88)


In [15]:
# def create_input(text_data):
#     input_sequences = []
#     for line in text_data:
#         token_list = tokenizer.texts_to_sequences([line])[0]
#         for i in range(1, len(token_list)):
#             n_gram_sequence = token_list[:i+1]
#             input_sequences.append(n_gram_sequence)
#     return input_sequences

# sequences_digit = create_input(sequences)
# print(sequences_digit[0:5])
# print(len(sequences_digit))

In [16]:
# # Tính toán độ dài tối đa của các chuỗi n-gram
# max_sequence_len = max([len(seq) for seq in sequences_digit])

# # Padding các chuỗi để có độ dài nhất định (max_sequence_len)
# input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(sequences_digit, maxlen=max_sequence_len, padding='pre'))

## **Build model**

In [17]:
model = TFAutoModel.from_pretrained("vinai/phobert-base")

tf_model.h5:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [18]:
# Định nghĩa vocab_size (số lượng từ trong từ điển)
vocab_size = tokenizer.vocab_size

# separate into input and output
X = input_ids[:, :-1]
y = input_ids[:, 1:]

In [19]:
# convert y to one-hot vector based on vocab (corpus)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

print('Shape of X:', X.shape)
print('Shape of y:', y.shape)

Shape of X: (14862, 87)
Shape of y: (14862, 87, 64000)


In [20]:
# Xây dựng mô hình Sequential
model = tf.keras.models.Sequential()

# Thêm các layer vào mô hình
model.add(tf.keras.layers.Input(shape=(max_length-1,), dtype=tf.int32))  # Input layer với shape tương ứng
model.add(model)  # Sử dụng PhoBERT model làm lớp trung gian
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))

# Hiển thị cấu trúc của mô hình
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


RecursionError: maximum recursion depth exceeded

In [ ]:


# Xây dựng mô hình Sequential
model = tf.keras.models.Sequential()

# Thêm các layer vào mô hình
model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=50, input_length=max_sequence_len-1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LSTM(512, return_sequences=True))
model.add(tf.keras.layers.LSTM(512))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))

# Hiển thị cấu trúc của mô hình
model.summary()

# Compile mô hình
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
summary = model.fit(X, y, batch_size=512, epochs=100)

In [18]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

2840

In [19]:
# separate into input and output
X = input_sequences[:,:-1]
y = input_sequences[:,-1]
# convert y to one-hot vector based on vocab (corpus)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [20]:
print(len(X[0]))
print(len(X))
print(X)

50
372595
[[   0    0    0 ...    0    0   26]
 [   0    0    0 ...    0   26   16]
 [   0    0    0 ...   26   16    6]
 ...
 [   0    0    0 ...    0  223  697]
 [   0    0    0 ...  223  697 2832]
 [   0    0    0 ...  697 2832  918]]


In [21]:
print('The length of a one-hot vector y: ', len(y[0]))
print('The collection of y vectors for all sentences', y)

The length of a one-hot vector y:  2840
The collection of y vectors for all sentences [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 50, input_length=50))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LSTM(512, return_sequences=True))
model.add(tf.keras.layers.LSTM(512))
model.add(tf.keras.layers.Dense(100, activation='relu'))
###
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax')) # can use linear for predict the index of word???
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            142000    
                                                                 
 batch_normalization (Batch  (None, 50, 50)            200       
 Normalization)                                                  
                                                                 
 lstm (LSTM)                 (None, 50, 512)           1153024   
                                                                 
 lstm_1 (LSTM)               (None, 512)               2099200   
                                                                 
 dense (Dense)               (None, 100)               51300     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                        

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
summary = model.fit(X, y, batch_size=512, epochs=100)

Epoch 1/100
728/728 [==============================] - 135s 174ms/step - loss: 3.8029 - accuracy: 0.3703
Epoch 2/100
728/728 [==============================] - 125s 172ms/step - loss: 1.6351 - accuracy: 0.6495
Epoch 3/100
728/728 [==============================] - 127s 174ms/step - loss: 0.7369 - accuracy: 0.8284
Epoch 4/100
728/728 [==============================] - 127s 175ms/step - loss: 0.4432 - accuracy: 0.8963
Epoch 5/100
728/728 [==============================] - 127s 174ms/step - loss: 0.3607 - accuracy: 0.9137
Epoch 6/100
728/728 [==============================] - 127s 174ms/step - loss: 0.3305 - accuracy: 0.9196
Epoch 7/100
728/728 [==============================] - 127s 174ms/step - loss: 0.3137 - accuracy: 0.9220
Epoch 8/100
613/728 [========================>.....] - ETA: 20s - loss: 0.2925 - accuracy: 0.9264

KeyboardInterrupt: 

In [ ]:
# test_loss, test_acc = model.evaluate(X, y, verbose=2)

## **Dump to model files**

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Text_Generator/model

/content/drive/My Drive/Colab Notebooks/Text_Generator/model


In [ ]:
import pickle
model.save('51_acc_language_model.h5')

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('sequences_digit.pkl', 'wb') as f:
    pickle.dump(sequences_digit, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
